## Import

In [1]:
import os
import pathlib
import pandas as pd

---

## Constants

In [2]:
OUTPUT_PATH = pathlib.Path("../../crawled-data")

In [3]:
BASE_TABLE_URL = "https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fsquads"

SHOOTING_TAIL      = "&div=div_stats_shooting_10730"
PASSING_TAIL       = "&div=div_stats_passing_10730"
PASSING_TYPES_TAIL = "&div=div_stats_passing_types_10730"
GCA_TAIL           = "&div=div_stats_gca_10730"
DEFENSE_TAIL       = "&div=div_stats_defense_10730"
POSSESSION_TAIL    = "&div=div_stats_possession_10730"
PLAYING_TIME_TAIL  = "&div=div_stats_playing_time_10730"
MISC_TAIL          = "&div=div_stats_misc_10730"

In [4]:
tables = {
    "italy": {
        "base_players": "https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F11%2Fstats%2FSerie-A-Stats&div=div_stats_standard",
        
        "Atalanta":      "%2F922493f3%2FAtalanta-Stats",
        "Benevento":     "%2F4fcb34fd%2FBenevento-Stats",
        "Bologna":       "%2F1d8099f8%2FBologna-Stats",
        "Cagliari":      "%2Fc4260e09%2FCagliari-Stats",
        "Crotone":       "%2F3074d7b1%2FCrotone-Stats",
        "Fiorentina":    "%2F421387cf%2FFiorentina-Stats",
        "Genoa":         "%2F658bf2de%2FGenoa-Stats",
        "Hellas Verona": "%2F0e72edf2%2FHellas-Verona-Stats",
        "Inter":         "%2Fd609edc0%2FInternazionale-Stats",
        "Juventus":      "%2Fe0652b02%2FJuventus-Stats",
        "Lazio":         "%2F7213da33%2FLazio-Stats",
        "Milan":         "%2Fdc56fe14%2FMilan-Stats",
        "Napoli":        "%2Fd48ad4ff%2FNapoli-Stats",
        "Parma":         "%2Feab4234c%2FParma-Stats",
        "Roma":          "%2Fcf74a709%2FRoma-Stats",
        "Sampdoria":     "%2F8ff9e3b3%2FSampdoria-Stats",
        "Sassuolo":      "%2Fe2befd26%2FSassuolo-Stats",
        "Spezia":        "%2F68449f6d%2FSpezia-Stats",
        "Torino":        "%2F105360fe%2FTorino-Stats",
        "Udinese":       "%2F04eea015%2FUdinese-Stats"
    }
}

In [6]:
base_players_colnames = {"Player": "name", "Nation": "nation", "Pos": "position", "Squad": "team",
                         "Age": "age", "Born": "born", "MP": "matches_played", "Starts": "matches_started",
                         "Min": "minutes_played", "90s": "mins_divide_90", "Gls": "goals", "Ast": "assists",
                         "G-PK": "non_penalty_goals", "PK": "penalty_goals", "PKatt": "penalty_attempts",
                         "CrdY": "yellow_cards", "CrdR": "red_cards", "xG": "xG", "npxG": "npxG", "xA": "xA"}

shooting_colnames = {"Player": "name", "Sh": "total_shots", "SoT": "shots_on_target",
                     "Dist": "avg_shot_distance", "FK": "free_kick_shots"}

passing_colnames = {"Player": "name", "Cmp": "completed_passes", "Att": "attempted_passes", "TotDist": "total_dist_passes",
                    "PrgDist": "prog_dist_passes", "Cmp.1": "completed_4m14m_passes", "Att.1": "attempted_4m14m_passes",
                    "Cmp.2": "completed_15m27m_passes", "Att.2": "attempted_15m27m_passes",
                    "Cmp.3": "completed_over27m_passes", "Att.3": "attempted_over27m_passes",
                    "KP": "assisted_shots", "1/3": "last_third_passes", "PPA": "passes_in_box", "CrsPA": "crosses_in_box"}

passing_types_colnames = {"Player": "name", "Live": "live_ball_passes", "Dead": "dead_ball_passes", "FK": "free_kick_passes",
                          "Press": "passes_under_pressure", "Sw": "passes_long_across_width", 
                          "Crs": "crosses", "CK": "corner_kicks", "Left": "passes_attempted_left_f",
                          "Right": "passes_attempted_right_f", "Head": "passes_attempted_h", 
                          "Off": "passes_to_offside", "Out": "passes_to_oob", "Int": "passes_intercepted", 
                          "Blocks": "passes_blocked_by_op"}

gca_colnames = {"Player": "name", "SCA": "shot_creating_actions", "SCA90": "shot_creating_actions_per_90s",
                "PassLive": "passes_live_lead_shot", "PassDead": "passes_dead_lead_shot",
                "Drib": "dribbles_lead_shot", "Sh": "shots_lead_shot", "Fld": "fouls_drawn_lead_shot",
                "Def": "def_actions_lead_shot", "GCA": "goal_creating_actions", "GCA90": "goal_creating_actions_per_90s",
                "PassLive.1": "passes_live_lead_goal", "PassDead.1": "passes_dead_lead_goal",
                "Drib.1": "dribbles_lead_goal", "Sh.1": "shots_lead_goal", "Fld.1": "fouls_drawn_lead_goal",
                "Def.1": "def_actions_lead_goal"}

defense_colnames = {"Player": "name", "Tkl": "tackles_total", "TklW": "tackles_win_possession",
                    "Def 3rd": "tackles_in_def_3rd", "Mid 3rd": "tackles_in_mid_3rd",
                    "Att 3rd": "tackles_in_att_3rd", "Press": "num_pressions_on_ball_holder",
                    "Succ": "num_poss_gain_after_5s_pressure", "Def 3rd.1": "num_pressions_def_3rd",
                    "Mid 3rd.1": "num_pressions_mid_3rd", "Att 3rd.1": "num_pressions_att_3rd",
                    "Blocks": "blocks_total", "Sh": "blocked_shots", "ShSv": "blocked_shots_on_goal",
                    "Pass": "blocked_passes", "Int": "interceptions", "Clr": "clearances",
                    "Err": "mistakes_lead_shot"}

possession_colnames = {"Player": "name", "Touches": "touches", "Def Pen": "touches_own_area",
                       "Def 3rd": "touches_def_3rd", "Mid 3rd": "touches_mid_3rd",
                       "Att 3rd": "touches_att_3rd", "Att Pen": "touches_opp_area",
                       "Succ": "dribbles_success", "Att": "dribbles_attempted",
                       "TotDist": "dist_with_ball", "PrgDist": "prog_dist_with_ball",
                       "1/3": "carries_in_last_3rd", "CPA": "carries_in_box",
                       "Targ": "pass_attempt_receiver", "Rec": "pass_success_receiver",
                       "Prog.1": "pass_receiver_prog"}

playing_time_colnames = {"Player": "name", "Mn/MP": "minutes_per_match",
                         "Subs": "num_sub_in", "unSub": "num_unused_sub",
                         "onG": "goals_scored_while_on_pitch", "onGA": "goals_allowed_while_on_pitch",
                         "onxG": "xG_while_on_pitch", "onxGA": "xG_allowed_while_on_pitch"}

misc_colnames = {"Player": "name", "Fls": "fouls_committed", "Fld": "fouls_drawn",
                 "Off": "offsides", "Won": "aerials_won", "Lost": "aerials_lost"}

In [8]:
base_players_coltypes = {"nation": str, "position": str, "team": str,
                         "mins_divide_90": float, "xG": float, "npxG": float, "xA": float}
shooting_coltypes = {"avg_shot_distance": float}
passing_coltypes = {"total_dist_passes": float, "prog_dist_passes": float}
passing_types_coltypes = {}
gca_coltypes = {}
defense_coltypes = {}
possession_coltypes = {"dist_with_ball": float, "prog_dist_with_ball": float}
playing_time_coltypes = {"xG_while_on_pitch": float, "xG_allowed_while_on_pitch": float}
misc_coltypes = {}

---

## Utilities

In [9]:
def get_table(url):
    return pd.read_html(url, header = 1, encoding = "utf-8")[0]

---

## Single Tables

In [56]:
def clean_base_players(df):
    df = df[df.Rk != "Rk"]
    df = df[list(base_players_colnames.keys())]
    df = df.rename(columns = base_players_colnames)

    coltypes = {col: float for col in list(base_players_colnames.values()) if col != "name"}
    coltypes.update(base_players_coltypes)
    df = df.astype(coltypes)
    
    df["nation"] = df["nation"].apply(lambda x: x.split(" ")[1])
    df["name"] = df["name"].apply(lambda x: x.encode("latin-1").decode("utf-8"))
    return df

In [57]:
def clean_shooting_table(df):
    df = df[list(shooting_colnames.keys())]
    df = df.rename(columns = shooting_colnames)
    
    coltypes = {col: float for col in list(shooting_colnames.values()) if col != "name"}
    coltypes.update(shooting_coltypes)
    df = df.astype(coltypes)
    
    df["name"] = df["name"].apply(lambda x: x.encode("latin-1").decode("utf-8"))
    df["avg_dist_shots"] = df["avg_dist_shots"].apply(lambda x: round(x*0.9144) if x > 0 else x)
    return df

In [58]:
def clean_passing_table(df):
    df = df[list(passing_colnames.keys())]
    df = df.rename(columns = passing_colnames)
    
    coltypes = {col: float for col in list(passing_colnames.values()) if col != "name"}
    coltypes.update(passing_coltypes)
    df = df.astype(coltypes)
    
    df["name"] = df["name"].apply(lambda x: x.encode("latin-1").decode("utf-8"))
    df["total_dist_passes"] = df["total_dist_passes"].apply(lambda x: round(x*0.9144) if x > 0 else x)
    df["prog_dist_passes"] = df["prog_dist_passes"].apply(lambda x: round(x*0.9144) if x > 0 else x)
    return df

In [59]:
def clean_passing_types_table(df):
    df = df[list(passing_types_colnames.keys())]
    df = df.rename(columns = passing_types_colnames)
    
    coltypes = {col: float for col in list(passing_types_colnames.values()) if col != "name"}
    coltypes.update(passing_types_coltypes)
    df = df.astype(coltypes)
    
    df["name"] = df["name"].apply(lambda x: x.encode("latin-1").decode("utf-8"))
    return df

In [60]:
def clean_gca_table(df):
    df = df[list(gca_colnames.keys())]
    df = df.rename(columns = gca_colnames)
    
    coltypes = {col: float for col in list(gca_colnames.values()) if col != "name"}
    coltypes.update(gca_coltypes)
    df = df.astype(coltypes)
    
    df["name"] = df["name"].apply(lambda x: x.encode("latin-1").decode("utf-8"))
    return df

In [61]:
def clean_defense_table(df):
    df = df[list(defense_colnames.keys())]
    df = df.rename(columns = defense_colnames)
    
    coltypes = {col: float for col in list(defense_colnames.values()) if col != "name"}
    coltypes.update(defense_coltypes)
    df = df.astype(coltypes)
    
    df["name"] = df["name"].apply(lambda x: x.encode("latin-1").decode("utf-8"))
    return df

In [62]:
def clean_possession_table(df):
    df = df[list(possession_colnames.keys())]
    df = df.rename(columns = possession_colnames)
    
    coltypes = {col: float for col in list(possession_colnames.values()) if col != "name"}
    coltypes.update(possession_coltypes)
    df = df.astype(coltypes)
    
    df["dist_with_ball"] = df["dist_with_ball"].apply(lambda x: round(x*0.9144) if x > 0 else x)
    df["prog_dist_with_ball"] = df["prog_dist_with_ball"].apply(lambda x: round(x*0.9144) if x > 0 else x)
    
    df["name"] = df["name"].apply(lambda x: x.encode("latin-1").decode("utf-8"))
    return df

In [63]:
def clean_playing_time_table(df):
    df = df[list(playing_time_colnames.keys())]
    df = df.rename(columns = playing_time_colnames)
    
    coltypes = {col: float for col in list(playing_time_colnames.values()) if col != "name"}
    coltypes.update(playing_time_coltypes)
    df = df.astype(coltypes)
    
    df["name"] = df["name"].apply(lambda x: x.encode("latin-1").decode("utf-8"))
    return df

In [64]:
def clean_misc_table(df):
    df = df[list(misc_colnames.keys())]
    df = df.rename(columns = misc_colnames)
    
    coltypes = {col: float for col in list(misc_colnames.values()) if col != "name"}
    coltypes.update(misc_coltypes)
    df = df.astype(coltypes)
    
    df["name"] = df["name"].apply(lambda x: x.encode("latin-1").decode("utf-8"))
    return df

---

## Team Functions

In [65]:
def get_base_players_data(competition):
    raw = get_table(url = tables[competition]["base_players"])
    df = clean_base_players(raw)
    return df

In [75]:
def get_all_teams_data(teams):
    dfs = list()
    for team in teams:
        dfs.append(get_team_data(team))
    df = pd.concat(dfs, ignore_index = True, axis = 1)
    return df

In [76]:
def get_team_data(team):
    shooting_url      = f"{BASE_TABLE_URL}{tables['italy'][team]}{SHOOTING_TAIL}"
    passing_url       = f"{BASE_TABLE_URL}{tables['italy']['Juventus']}{PASSING_TAIL}"
    passing_types_url = f"{BASE_TABLE_URL}{tables['italy']['Juventus']}{PASSING_TYPES_TAIL}"
    gca_url           = f"{BASE_TABLE_URL}{tables['italy']['Juventus']}{GCA_TAIL}"
    defense_url       = f"{BASE_TABLE_URL}{tables['italy']['Juventus']}{DEFENSE_TAIL}"
    possession_url    = f"{BASE_TABLE_URL}{tables['italy']['Juventus']}{POSSESSION_TAIL}"
    playing_time_url  = f"{BASE_TABLE_URL}{tables['italy']['Juventus']}{PLAYING_TIME_TAIL}"
    misc_url          = f"{BASE_TABLE_URL}{tables['italy']['Juventus']}{MISC_TAIL}"
    
    df_shoot        = get_table(url = shooting_url)
    print(f"Downloaded Shooting Table for {team}")
    
    df_pass         = get_table(url = passing_url)
    print(f"Downloaded Passing Table for {team}")
    
    df_pass_types   = get_table(url = passing_types_url)
    print(f"Downloaded Passing Types Table for {team}")
    
    df_gca          = get_table(url = gca_url)
    print(f"Downloaded GCA Table for {team}")
    
    df_defense      = get_table(url = defense_url)
    print(f"Downloaded Defense Table for {team}")
    
    df_possession   = get_table(url = possession_url)
    print(f"Downloaded Possession Table for {team}")
    
    df_playing_time = get_table(url = playing_time_url)
    print(f"Downloaded Playing Time Table for {team}")
    
    df_misc         = get_table(url = misc_url)
    print(f"Downloaded Misc Table for {team}")
    
    df_shoot        = clean_shooting_table(df_shoot)
    df_pass         = clean_passing_table(df_pass)
    df_pass_types   = clean_passing_types_table(df_pass_types)
    df_gca          = clean_gca_table(df_gca)
    df_defense      = clean_defense_table(df_defense)
    df_possession   = clean_possession_table(df_possession)
    df_playing_time = clean_playing_time_table(df_playing_time)
    df_misc         = clean_misc_table(df_misc)
    
    df_team = df_shoot.merge(df_pass,         on = "name", how = "inner")\
                      .merge(df_pass_types,   on = "name", how = "inner")\
                      .merge(df_gca,          on = "name", how = "inner")\
                      .merge(df_defense,      on = "name", how = "inner")\
                      .merge(df_possession,   on = "name", how = "inner")\
                      .merge(df_playing_time, on = "name", how = "inner")\
                      .merge(df_misc,         on = "name", how = "inner")
    print(f"[OK] {team}")
    return df_team

---

In [77]:
competition = "italy"

In [78]:
base_players = get_base_players_data(competition = "italy")

In [79]:
teams = [key for key in list(tables[competition].keys()) if key != "base_players"]

In [80]:
teams_df = get_all_teams_data(teams)

Downloaded Shooting Table for Atalanta
Downloaded Passing Table for Atalanta
Downloaded Passing Types Table for Atalanta
Downloaded GCA Table for Atalanta
Downloaded Defense Table for Atalanta
Downloaded Possession Table for Atalanta
Downloaded Playing Time Table for Atalanta
Downloaded Misc Table for Atalanta
[OK] Atalanta
Downloaded Shooting Table for Benevento
Downloaded Passing Table for Benevento
Downloaded Passing Types Table for Benevento
Downloaded GCA Table for Benevento
Downloaded Defense Table for Benevento
Downloaded Possession Table for Benevento
Downloaded Playing Time Table for Benevento
Downloaded Misc Table for Benevento
[OK] Benevento
Downloaded Shooting Table for Bologna
Downloaded Passing Table for Bologna
Downloaded Passing Types Table for Bologna
Downloaded GCA Table for Bologna
Downloaded Defense Table for Bologna
Downloaded Possession Table for Bologna
Downloaded Playing Time Table for Bologna
Downloaded Misc Table for Bologna
[OK] Bologna
Downloaded Shooting Ta

In [81]:
len(base_players)

619

In [82]:
len(teams_df)

32

---

## Serie A 2020-2021 Base Players

In [90]:
raw = get_table(url = tables["italy"]["base_players"])

In [93]:
df = clean_base_players(raw)

In [94]:
df.tail()

,name,nation,position,team,age,born,matches_played,matches_started,minutes_played,mins_divide_90,goals,assists,non_penalty_goals,penalty_goals,penalty_attempts,yellow_cards,red_cards,xG,npxG,xA
638,Marvin Zeegelaar,NED,DF,Udinese,29,1990,24,23,1740,19.3,1,0,1,0,0,6,0,1.5,1.5,1.1
639,Piotr Zieliński,POL,MF,Napoli,26,1994,36,32,2523,28.0,8,10,8,0,0,2,0,5.4,5.4,5.4
640,Joshua Zirkzee,NED,"FW,MF",Parma,19,2001,4,0,113,1.3,0,0,0,0,0,0,0,0.1,0.1,0.2
641,Jeroen Zoet,NED,GK,Spezia,29,1991,7,7,603,6.7,0,0,0,0,0,1,0,0.0,0.0,0.0
642,Filip Đuričić,SRB,"MF,FW",Sassuolo,28,1992,32,27,2078,23.1,5,4,4,1,1,5,0,5.7,5.0,6.1


---

## Shooting

In [95]:
shooting_url = f"{BASE_TABLE_URL}{tables['italy']['Juventus']}{SHOOTING_TAIL}"

In [96]:
df = get_table(url = shooting_url)

In [97]:
df = clean_shooting_table(df)

In [98]:
df.head()

,name,total_shots,shots_on_target,avg_dist_shots,free_kick_shots
0,Danilo,18,2,19.0,0
1,Cristiano Ronaldo,162,60,16.0,17
2,Wojciech Szczęsny,0,0,NaN,0
3,Federico Chiesa,60,25,16.0,0
4,Rodrigo Bentancur,15,5,21.0,0


## Passing

In [99]:
passing_url = f"{BASE_TABLE_URL}{tables['italy']['Juventus']}{PASSING_TAIL}"

In [100]:
df = get_table(url = passing_url)

In [101]:
df = clean_passing_table(df)

In [102]:
df.head()

,name,completed_passes,attempted_passes,total_dist_passes,prog_dist_passes,completed_4m14m_passes,attempted_4m14m_passes,completed_15m27m_passes,attempted_15m27m_passes,completed_over27m_passes,attempted_over27m_passes,assisted_shots,last_third_passes,passes_in_box,crosses_in_box
0,Danilo,2194,2484,40372,12503,736,800,1137,1238,305,406,27,228,33,9
1,Cristiano Ronaldo,853,1086,11884,2583,485,575,289,357,55,87,30,59,32,6
2,Wojciech Szczęsny,910,1015,21304,11046,186,186,444,449,279,376,1,4,0,0
3,Federico Chiesa,626,808,10254,3186,303,354,235,293,77,118,45,45,41,17
4,Rodrigo Bentancur,1651,1851,29633,7452,615,668,782,841,234,298,30,168,19,5


## Passing Types

In [103]:
passing_types_url = f"{BASE_TABLE_URL}{tables['italy']['Juventus']}{PASSING_TYPES_TAIL}"

In [104]:
df = get_table(url = passing_types_url)

In [105]:
df = clean_passing_types_table(df)

In [106]:
df.head()

,name,live_ball_passes,dead_ball_passes,free_kick_passes,passes_under_pressure,passes_long_across_width,crosses,corner_kicks,passes_attempted_left_f,passes_attempted_right_f,passes_attempted_h,passes_to_offside,passes_to_oob,passes_intercepted,passes_blocked_by_op
0,Danilo,2286,198,43,217,42,28,2,304,1955,44,5,0,51,33
1,Cristiano Ronaldo,1033,53,14,191,23,52,3,160,859,24,6,0,38,42
2,Wojciech Szczęsny,803,212,32,77,28,0,0,117,757,1,1,0,3,5
3,Federico Chiesa,754,54,6,114,27,62,16,147,603,21,1,8,25,33
4,Rodrigo Bentancur,1779,72,50,275,53,8,16,373,1396,50,5,8,32,28


## GCA

In [36]:
gca_url = f"{BASE_TABLE_URL}{tables['italy']['Juventus']}{GCA_TAIL}"

In [37]:
df = get_table(url = gca_url)

In [38]:
df = clean_gca_table(df)

In [39]:
df.head()

,name,shot_creating_actions,shot_creating_actions_per_90s,passes_live_lead_shot,passes_dead_lead_shot,dribbles_lead_shot,shots_lead_shot,fouls_drawn_lead_shot,def_actions_lead_shot,goal_creating_actions,goal_creating_actions_per_90s,passes_live_lead_goal,passes_dead_lead_goal,dribbles_lead_goal,shots_lead_goal,fouls_drawn_lead_goal,def_actions_lead_goal
0,Danilo,60,1,57,1,1,1,0,0,5,0,5,0,0,0,0,0
1,Cristiano Ronaldo,114,3,62,1,18,17,12,4,16,0,7,0,3,5,1,0
2,Wojciech Szczęsny,2,0,2,0,0,0,0,0,2,0,2,0,0,0,0,0
3,Federico Chiesa,86,3,60,6,8,6,2,4,16,0,11,0,2,1,1,1
4,Rodrigo Bentancur,66,2,50,9,0,1,1,5,13,0,11,0,0,0,0,2


## Defense

In [112]:
defense_url = f"{BASE_TABLE_URL}{tables['italy']['Juventus']}{DEFENSE_TAIL}"

In [113]:
df = get_table(url = defense_url)

In [114]:
df = clean_defense_table(df)

In [115]:
df.head()

,name,tackles_total,tackles_win_possession,tackles_in_def_3rd,tackles_in_mid_3rd,tackles_in_att_3rd,num_pressions_on_ball_holder,num_poss_gain_after_5s_pressure,num_pressions_def_3rd,num_pressions_mid_3rd,num_pressions_att_3rd,blocks_total,blocked_shots,blocked_shots_on_goal,blocked_passes,interceptions,clearances,mistakes_lead_shot
0,Danilo,60,37,39,13,8,249,80,134,85,30,54,13,0,41,44,82,0
1,Cristiano Ronaldo,9,3,3,5,1,225,61,24,89,112,15,5,0,10,8,16,0
2,Wojciech Szczęsny,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0
3,Federico Chiesa,58,40,24,20,14,397,131,137,147,113,27,1,0,26,11,21,0
4,Rodrigo Bentancur,74,45,27,38,9,537,158,169,274,94,56,16,0,40,48,45,1


## Possession

In [120]:
possession_url = f"{BASE_TABLE_URL}{tables['italy']['Juventus']}{POSSESSION_TAIL}"

In [121]:
df = get_table(url = possession_url)

In [128]:
df = clean_possession_table(df)

In [129]:
df.head()

,name,touches,touches_own_area,touches_def_3rd,touches_mid_3rd,touches_att_3rd,touches_opp_area,dribbles_success,dribbles_attempted,dist_with_ball,prog_dist_with_ball,carries_in_last_3rd,carries_in_box,pass_attempt_receiver,pass_success_receiver,pass_receiver_prog
0,Danilo,2747,162,909,1577,416,24,20,22,7971,4505.0,43,5,2077,2025,29
1,Cristiano Ronaldo,1462,19,75,675,846,213,58,90,5890,3830.0,85,59,1678,1207,291
2,Wojciech Szczęsny,1076,935,1066,10,0,0,2,2,2411,1317.0,0,0,607,606,0
3,Federico Chiesa,1100,22,165,457,579,122,35,62,4675,3046.0,45,44,1006,789,132
4,Rodrigo Bentancur,2122,115,564,1303,385,15,25,39,6386,3029.0,40,4,1585,1509,35


## Playing Time

In [132]:
playing_time_url = f"{BASE_TABLE_URL}{tables['italy']['Juventus']}{PLAYING_TIME_TAIL}"

In [146]:
df = get_table(url = playing_time_url)

In [147]:
df = clean_playing_time_table(df)

In [148]:
df.head()

,name,minutes_per_match,num_sub_in,num_unused_sub,goals_scored_while_on_pitch,goals_allowed_while_on_pitch,xG_while_on_pitch,xG_allowed_while_on_pitch
0,Danilo,86.0,2.0,3.0,69.0,30.0,64.2,33.0
1,Cristiano Ronaldo,85.0,2.0,1.0,64.0,29.0,60.8,31.9
2,Wojciech Szczęsny,89.0,0.0,8.0,55.0,32.0,54.5,28.9
3,Federico Chiesa,74.0,2.0,1.0,50.0,25.0,47.4,22.7
4,Rodrigo Bentancur,70.0,6.0,1.0,48.0,25.0,50.2,24.0


## Misc

In [19]:
misc_url = f"{BASE_TABLE_URL}{tables['italy']['Juventus']}{MISC_TAIL}"

In [20]:
df = get_table(url = misc_url)

In [21]:
df = clean_misc_table(df)

In [22]:
df.head()

,name,fouls_committed,fouls_drawn,offsides,aerials_won,aerials_lost
0,Danilo,25.0,20.0,2.0,34.0,18.0
1,Cristiano Ronaldo,17.0,43.0,31.0,29.0,21.0
2,Wojciech Szczęsny,0.0,1.0,0.0,0.0,0.0
3,Federico Chiesa,30.0,36.0,11.0,6.0,34.0
4,Rodrigo Bentancur,45.0,28.0,2.0,31.0,26.0
